# BAU Evaluation and elements of concern 

author: Katherine Shaw

Date: August 7 - August 15 

Purpose of Document: the role of this document is to validated the 2020 year produced by the model to actual entoso-e and EU data to ensure the model's behavior aligns with realistic developments, and if not, answer why. 

Any new elements of model behavior that were of concern across the project will also be enumerated and evaluated, unless said problem requires extensive model runs to evaluate. 

TDLR: gas is being underutilized, and this notebook is the clean version dedicated to finding out why. Ultimately, there will elements that I could change to make it better, but the lack of gas usage in the sector and electric only models continues to be a problem. Without aid from an outside source, I've exhausted my ideas for it, and will enforce an operational constraint to use gas if I cannot find the right configuration file to replicate 2020 with an order-of-magnitude similarity. 

In [2]:
#Packages 
import pypsa
import matplotlib.pyplot as plt 
import cartopy 
import geopandas
import networkx
import linopy
import cartopy.crs as ccrs
import atlite 
import geopandas as gpd 
import xarray
import pandas as pd 
from datetime import datetime
import numpy as np
from pypsa.plot import add_legend_patches
import random
import plotly.graph_objects as go

## Overall view of colors matched to carriers 
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

#importing auxillary functions
import Auxillary_Functions as af

- build monthly prices 
- DLR 
- more weather years 
- carbon stores vs carbon sequestration 
- dac vs no dac done elsewhere 

# At first started this because I wanted to match hydrogen electorlysis and SMR usage to their 2020 counterparts 

### How many European Hydrogen Projects exist and what is their capacity? 

In [22]:
hydrogen_projects = pd.ExcelFile('/Users/katherine.shaw/Desktop/IEA_Hydrogen_Production_Projects_Database_update_5_March_2025.xlsx')
hydrogen_projects.sheet_names
hydrogen_projects = pd.read_excel('/Users/katherine.shaw/Desktop/IEA_Hydrogen_Production_Projects_Database_update_5_March_2025.xlsx', engine = 'openpyxl', sheet_name = 'Projects', header = [0,3])
hydrogen_projects.head(10)
 #the first six rows are actually summary headers, just read in oddly, and not actual projects
European_filtered_country_to_code = {
    'Albania': 'ALB',
    'Austria': 'AUT',
    'Belgium': 'BEL',
    'Bulgaria': 'BGR',
    'Bosnia and Herzegovina': 'BIH',
    'Switzerland': 'CHE',
    'Czechia': 'CZE',
    'Germany': 'DEU',
    'Denmark': 'DNK',
    'Spain': 'ESP',
    'Estonia': 'EST',
    'Finland': 'FIN',
    'France': 'FRA',
    'United Kingdom': 'GBR',
    'Greece': 'GRC',
    'Croatia': 'HRV',
    'Hungary': 'HUN',
    'Ireland': 'IRL',
    'Italy': 'ITA',
    'Lithuania': 'LTU',
    'Luxembourg': 'LUX',
    'Latvia': 'LVA',
    'North Macedonia': 'MKD',
    'Montenegro': 'MNE',
    'Netherlands': 'NLD',
    'Norway': 'NOR',
    'Poland': 'POL',
    'Portugal': 'PRT',
    'Romania': 'ROU',
    'Serbia': 'SRB',
    'Slovak Republic': 'SVK',
    'Slovenia': 'SVN',
    'Sweden': 'SWE',
    'Kosovo': 'XKX'
}
len(European_filtered_country_to_code)


/Users/katherine.shaw/miniconda3/envs/pypsa-eur/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning:

Data Validation extension is not supported and will be removed



34

In [53]:
European_hydrogen_projects = hydrogen_projects[hydrogen_projects[('DATABASE', 'Country')].isin(European_filtered_country_to_code.values())].copy()
pre_2023 = European_hydrogen_projects[European_hydrogen_projects[('DATABASE', 'Date online')] <= 2023].copy()
pre_2023[('DATABASE', 'Technology')].unique()
pre_2023 = pre_2023[pre_2023[('DATABASE', 'Technology')] == 'NG w CCUS']#,'Oil w CCUS', 'Other'])]
pre_2023#[('DATABASE', 'Capacity_MWel')].sum()
#gives 271 in electrolyzer capacity, 
#no SMR because it only lists smr with carbon capture 

DATABASE                                                        \
         Ref     Project name Country Date online Decomission date   
372    710.0      Port Jerome     FRA      2015.0              NaN   
670   1081.0  Sapio - Mantova     ITA      2016.0              NaN   

                                                                       \
          Status Technology Technology_details Technology_electricity   
372  Operational  NG w CCUS           SMR+CCUS                    NaN   
670  Operational  NG w CCUS           SMR+CCUS                    NaN   

                                    ...                                  \
    Technology_electricity_details  ... Capacity_MWel Capacity_Nm³ H₂/h   
372                            NaN  ...           NaN           50000.0   
670                            NaN  ...           NaN            1500.0   

                                                \
    Capacity_kt H2/y Capacity_t CO₂ captured/y   
372         38.98200                  100000.0   
670          1.16946                       NaN   

                                                                              \
    IEA zero-carbon estimated normalized capacity [Nm³ H₂/hour]   References   
372                                       12537.644277           [335] [336]   
670                                           0.000000                   NaN   

                                                           
      Latitude  Longitude                 Dummy_1 LOWE_CF  
372  49.483790   0.548356  Fossil fuels with CCUS     0.9  
670  45.155733  10.789729  Fossil fuels with CCUS     0.9  

[2 rows x 36 columns]

Will be hard to get SMR knowledge via any method because SMR output is given in terms of hydrogen produced, not regularly in terms of capacity MW or GW like electrolyzers. If SMR and SMR ccus is brought into pypsa as part of a conventional energy source, then I don't need to worry, 

checking that now  

In [76]:
network_2020 = pypsa.Network('/Users/katherine.shaw/Desktop/pypsa-eur/resources/networks/base_s_39___2020.nc')
network_2020.links.carrier.unique() #SMR, SMR CC 
network_2020.links[network_2020.links.carrier.isin(['SMR', 'SMR CC'])]
network_2020.statistics().loc['Link']

INFO:pypsa.io:Imported network base_s_39___2020.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


,Optimal Capacity,Installed Capacity,Supply,Withdrawal,Energy Balance,Transmission,Capacity Factor,Curtailment,Capital Expenditure,Operational Expenditure,Revenue,Market Value
DC,0.0,5.404000e+04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
gas pipeline,0.0,2.563504e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
oil refining,0.0,1.000000e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
unsustainable bioliquids,0.0,1.921633e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
network_2020 = pypsa.Network('/Users/katherine.shaw/Desktop/pypsa-eur/resources/networks/base_s_39___2020_brownfield.nc')
network_2020.links.carrier.unique() #SMR, SMR CC 
network_2020.links[network_2020.links.carrier.isin(['SMR', 'SMR CC'])]
network_2020.statistics().loc['Link']

INFO:pypsa.io:Imported network base_s_39___2020_brownfield.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


,Optimal Capacity,Installed Capacity,Supply,Withdrawal,Energy Balance,Transmission,Capacity Factor,Curtailment,Capital Expenditure,Operational Expenditure,Revenue,Market Value
Combined-Cycle Gas,0.0,3.721534e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DC,0.0,5.404000e+04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Open-Cycle Gas,0.0,5.827500e+01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
coal,0.0,2.817420e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
gas pipeline,0.0,2.563504e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
lignite,0.0,1.838664e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
nuclear,0.0,3.914543e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
oil,0.0,1.721052e+04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
oil refining,0.0,1.000000e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
residential rural resistive heater,0.0,5.296723e+04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
network_2020 = pypsa.Network('/Users/katherine.shaw/Desktop/pypsa-eur/results/networks/base_s_39___2020.nc')
network_2020.links.carrier.unique() #SMR, SMR CC 
network_2020.links[network_2020.links.carrier.isin(['H2 Electrolysis'])]
#network_2020.statistics().loc['Link'].loc['SMR']

INFO:pypsa.io:Imported network base_s_39___2020.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


,bus0,bus1,type,carrier,efficiency,active,build_year,lifetime,p_nom,p_nom_mod,...,under_construction,reversed,tags,length_original,voltage,underwater_fraction,location,project_status,geometry,dc
Link,,,,,,,,,,,,,,,,,,,,,
AL0 0 H2 Electrolysis-2020,AL0 0,AL0 0 H2,,H2 Electrolysis,0.5773,True,2020,25.0,0.0,0.0,...,NaN,False,,0.0,NaN,NaN,,,,NaN
AT0 0 H2 Electrolysis-2020,AT0 0,AT0 0 H2,,H2 Electrolysis,0.5773,True,2020,25.0,0.0,0.0,...,NaN,False,,0.0,NaN,NaN,,,,NaN
BA0 0 H2 Electrolysis-2020,BA0 0,BA0 0 H2,,H2 Electrolysis,0.5773,True,2020,25.0,0.0,0.0,...,NaN,False,,0.0,NaN,NaN,,,,NaN
BE0 0 H2 Electrolysis-2020,BE0 0,BE0 0 H2,,H2 Electrolysis,0.5773,True,2020,25.0,0.0,0.0,...,NaN,False,,0.0,NaN,NaN,,,,NaN
BG0 0 H2 Electrolysis-2020,BG0 0,BG0 0 H2,,H2 Electrolysis,0.5773,True,2020,25.0,0.0,0.0,...,NaN,False,,0.0,NaN,NaN,,,,NaN
CH0 0 H2 Electrolysis-2020,CH0 0,CH0 0 H2,,H2 Electrolysis,0.5773,True,2020,25.0,0.0,0.0,...,NaN,False,,0.0,NaN,NaN,,,,NaN
CZ0 0 H2 Electrolysis-2020,CZ0 0,CZ0 0 H2,,H2 Electrolysis,0.5773,True,2020,25.0,0.0,0.0,...,NaN,False,,0.0,NaN,NaN,,,,NaN
DE0 0 H2 Electrolysis-2020,DE0 0,DE0 0 H2,,H2 Electrolysis,0.5773,True,2020,25.0,0.0,0.0,...,NaN,False,,0.0,NaN,NaN,,,,NaN
DK0 0 H2 Electrolysis-2020,DK0 0,DK0 0 H2,,H2 Electrolysis,0.5773,True,2020,25.0,0.0,0.0,...,NaN,False,,0.0,NaN,NaN,,,,NaN


In [80]:
network_2020.statistics().loc['Link'].loc['SMR CC']

Optimal Capacity           1.388433e+03
Installed Capacity         0.000000e+00
Supply                     1.080018e+07
Withdrawal                 1.216236e+07
Energy Balance            -1.362184e+06
Transmission               0.000000e+00
Capacity Factor            9.999746e-01
Curtailment                0.000000e+00
Capital Expenditure        1.098294e+08
Operational Expenditure    1.332266e+05
Revenue                    1.099572e+08
Market Value               1.018105e+01
Name: SMR CC, dtype: float64

No SMR capacity is installed in a brownfield manner, it's added in during the greenfield expansion  

In [87]:
network_2020.links[network_2020.links.carrier.isin(['SMR', 'SMR CC'])]
network_2020.buses.loc['PT0 0'] #the unit at the end of the hydrogen pipelines ins MWh_LHV 
#going from MWh to MWH trend

v_nom                 380.0
type                       
x                 -8.248102
y                 40.187334
carrier                  AC
unit                 MWh_el
v_mag_pu_set            1.0
v_mag_pu_min            0.0
v_mag_pu_max            inf
control                  PQ
generator                  
sub_network                
substation_lv           1.0
location              PT0 0
country                  PT
substation_off          1.0
Name: PT0 0, dtype: object

In [88]:
network_2020.links[network_2020.links.carrier.isin(['SMR', 'SMR CC'])].loc['AL0 0 SMR-2020']


bus0                            AL0 0 gas
bus1                             AL0 0 H2
type                                     
carrier                               SMR
efficiency                           0.76
active                               True
build_year                           2020
lifetime                             30.0
p_nom                                 0.0
p_nom_mod                             0.0
p_nom_extendable                     True
p_nom_min                             0.0
p_nom_max                             inf
p_set                                 0.0
p_min_pu                              0.0
p_max_pu                              1.0
capital_cost                 68192.356925
marginal_cost                    0.009036
marginal_cost_quadratic               0.0
stand_by_cost                         0.0
length                                0.0
terrain_factor                        1.0
committable                         False
start_up_cost                     